In this notebook, we will be trying to build an algorithmic trading bot, that uses a combination of machine learning models for sentiment analysis, fundamental economic news for real time decision making, and technical analysis strategies for a general market structure for precise entry and exit of trades.

SECTION 1: CONNECTING OUR BOT TO OUR MT5 ACCOUNT

In [3]:
# # install all our required python libraries
# !pip3 install MetaTrader5
# !pip install numpy==1.26.4
# !pip install pandas

In [2]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import time

In [1]:
# declare our mt5 credentials.
# Note: these should be stored in a different file and imported
login = 25103183
password = 'jV}hv(}+138S'
server = 'TickmillUK-Demo'

In [9]:
# # declare our mt5 credentials.
# # Note: these should be stored in a different file and imported
# login = 83897010
# password = '!6VjlhTi'
# server = 'MetaQuotes-Demo'

In [4]:
# function to login to mt5 and check if it was successful

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =",mt5.last_error())
    quit()
authorized=mt5.login(login, password=password, server=server)

if authorized:
    print('MT5 Account Logged In : ', mt5.account_info().trade_allowed)
else:
    print("failed to connect at login #{}, error code: {}".format(login, mt5.last_error()))
    mt5.shutdown()

MT5 Account Logged In :  True


In [6]:
# get mt5 account info
account_info = mt5.account_info()
num_symbols = mt5.symbols_total()
print(datetime.now(),
              '| Login: ', account_info.login,
              '| Balance: ', account_info.balance,
              '| Equity: ' , account_info.equity,
              '| Number of symbols: ', num_symbols)


2024-07-08 09:20:41.382252 | Login:  25103183 | Balance:  9797.47 | Equity:  9797.24 | Number of symbols:  618


SECTION 2: EXTRACT CANDLESTICK DATA FROM A PARTICULAR TRADING PAIR

In [6]:
# get the open, close, hgh, and close prices of a pair on a particular timeframe within a time period
# display the data using the pandas library
minute_candle = pd.DataFrame(mt5.copy_rates_range('GBPUSD',mt5.TIMEFRAME_M1, datetime(2024,6,21), datetime.now()))
minute_candle.head()

,time,open,high,low,close,tick_volume,spread,real_volume
0,1718924400,1.26631,1.26631,1.26624,1.26628,61,3,0
1,1718924460,1.26628,1.26631,1.26626,1.26630,19,3,0
2,1718924520,1.26630,1.26636,1.26630,1.26632,25,3,0
3,1718924580,1.26633,1.26633,1.26631,1.26632,3,3,0
4,1718924640,1.26632,1.26634,1.26628,1.26630,16,3,0


In [7]:
# declare the parameters of the candlestick data you want to get
ticker = 'GBPUSD' # currency pair we will be extracting from
interval = mt5.TIMEFRAME_D1 # daily candlestick
from_date = datetime.now() # current time
no_of_rows = 100
qty = 1

daily_candle = mt5.copy_rates_from(ticker, interval, from_date, no_of_rows)
# convert to a pandas dataframe
daily_candle_df = pd.DataFrame(daily_candle)
daily_candle_df.head()

,time,open,high,low,close,tick_volume,spread,real_volume
0,1708387200,1.25905,1.26687,1.25794,1.26221,135687,0,0
1,1708473600,1.26157,1.26419,1.26026,1.26374,141493,0,0
2,1708560000,1.26310,1.27096,1.26116,1.26607,172131,0,0
3,1708646400,1.26513,1.27014,1.26486,1.26699,126453,0,0
4,1708905600,1.26715,1.26994,1.26566,1.26853,93609,0,0


In [21]:
# Change the time column to a per second format
minute_candle['time'] = pd.to_datetime(minute_candle['time'],unit='s')
minute_candle.head()

,time,open,high,low,close,tick_volume,spread,real_volume
0,2024-06-20 23:00:00,1.26631,1.26631,1.26624,1.26628,61,3,0
1,2024-06-20 23:01:00,1.26628,1.26631,1.26626,1.26630,19,3,0
2,2024-06-20 23:02:00,1.26630,1.26636,1.26630,1.26632,25,3,0
3,2024-06-20 23:03:00,1.26633,1.26633,1.26631,1.26632,3,3,0
4,2024-06-20 23:04:00,1.26632,1.26634,1.26628,1.26630,16,3,0


TEST BUY AND SELL ORDER

In [8]:
import time
 
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)
 
 
# prepare the buy request structure
symbol = "GBPUSD"
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(symbol, "not found, can not call order_check()")
    mt5.shutdown()
    quit()
 
# if the symbol is unavailable in MarketWatch, add it
if not symbol_info.visible:
    print(symbol, "is not visible, trying to switch on")
    if not mt5.symbol_select(symbol,True):
        print("symbol_select({}}) failed, exit",symbol)
        mt5.shutdown()
        quit()
 
lot = 0.01
point = mt5.symbol_info(symbol).point
price = mt5.symbol_info_tick(symbol).ask
deviation = 20
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY,
    "price": price,
    "sl": price - 100 * point,
    "tp": price + 100 * point,
    "deviation": deviation,
    "magic": 234000,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
}
 
# send a trading request
result = mt5.order_send(request)
# check the execution result
print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
if result.retcode != mt5.TRADE_RETCODE_DONE:
    print("2. order_send failed, retcode={}".format(result.retcode))
    # request the result as a dictionary and display it element by element
    result_dict=result._asdict()
    for field in result_dict.keys():
        print("   {}={}".format(field,result_dict[field]))
        # if this is a trading request structure, display it element by element as well
        if field=="request":
            traderequest_dict=result_dict[field]._asdict()
            for tradereq_filed in traderequest_dict:
                print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
    print("shutdown() and quit")
    # mt5.shutdown()
    # quit()
 
print("2. order_send done, ", result)
print("   opened position with POSITION_TICKET={}".format(result.order))
print("   sleep 10 seconds before closing position #{}".format(result.order))
time.sleep(10)


# create a close request
position_id=result.order
price=mt5.symbol_info_tick(symbol).bid
deviation=20
request={
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL,
    "position": position_id,
    "price": price,
    "deviation": deviation,
    "magic": 234000,
    "comment": "python script close",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
}
# send a trading request
result=mt5.order_send(request)
# check the execution result
print("3. close position #{}: sell {} {} lots at {} with deviation={} points".format(position_id,symbol,lot,price,deviation));
if result.retcode != mt5.TRADE_RETCODE_DONE:
    print("4. order_send failed, retcode={}".format(result.retcode))
    print("   result",result)
else:
    print("4. position #{} closed, {}".format(position_id,result))
    # request the result as a dictionary and display it element by element
    result_dict=result._asdict()
    for field in result_dict.keys():
        print("   {}={}".format(field,result_dict[field]))
        # if this is a trading request structure, display it element by element as well
        if field=="request":
            traderequest_dict=result_dict[field]._asdict()
            for tradereq_filed in traderequest_dict:
                print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
 
# shut down connection to the MetaTrader 5 terminal
# mt5.shutdown()

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4424
1. order_send(): by GBPUSD 0.01 lots at 1.28162 with deviation=20 points
2. order_send done,  OrderSendResult(retcode=10009, deal=37681308, order=45457686, volume=0.01, price=1.28163, bid=0.0, ask=0.0, comment='Request executed', request_id=3368069444, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='GBPUSD', volume=0.01, price=1.28162, stoplimit=0.0, sl=1.28062, tp=1.2826199999999999, deviation=20, type=0, type_filling=1, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))
   opened position with POSITION_TICKET=45457686
   sleep 2 seconds before closing position #45457686
3. close position #45457686: sell GBPUSD 0.01 lots at 1.28143 with deviation=20 points
4. position #45457686 closed, OrderSendResult(retcode=10009, deal=37681311, order=45457689, volume=0.01, price=1.28143, bid=0.0, ask=0.0, comment='Request executed', request_id=33680

SECTION 3: IMPLEMENTING SIMPLE TECHNICAL ANALYSIS FOR BUY OR SELL ORDER

In [9]:
symbol = 'GBPUSD' # British pounds/US dollar
timeframe = mt5.TIMEFRAME_M5 # for 5-minute intervals
volume = 0.01 # lot size
strategy_name = 'ma_trendfollowing' # feel free to name it as you want

In [10]:
# get the simple moving average for a given symbol, timeframe and period
# Use mt5.copy_rates_from_pos to retrieve recent price data
# Calculate the mean of the closing prices for the specified period 
def get_sma(symbol, timeframe, period):
    sma = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, 1, period))['close'].mean()

    return sma

In [12]:
# Closes an existing trading position based on the provided position details.
# Uses order_type_dict and price_dict to determine order type and price based on the position type (buy or sell).
# Defines an order request with details like symbol, volume, and magic number.
# Sends the close order request to the MT5 platform using mt5.order_send.
# Returns the order execution result.

def close_position(position, deviation=20, magic=12345):

    order_type_dict = {
        0: mt5.ORDER_TYPE_SELL,
        1: mt5.ORDER_TYPE_BUY
    }

    price_dict = {
        0: mt5.symbol_info_tick(symbol).bid,
        1: mt5.symbol_info_tick(symbol).ask
    }

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "position": position['ticket'],  # select the position you want to close
        "symbol": symbol,
        "volume": volume,  # FLOAT
        "type": order_type_dict[position['type']],
        "price": price_dict[position['type']],
        "deviation": deviation,  # INTERGER
        "magic": magic,  # INTERGER
        "comment": strategy_name,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order_result = mt5.order_send(request)
    return(order_result)

In [13]:
# Closes all open positions based on the specified order type ('buy', 'sell', or 'all').
# Retrieves all open positions using mt5.positions_get.
# Converts the positions data to a Pandas DataFrame for easier manipulation.
# Filters positions based on the order_type if provided (closes only buys or sells).
# Loops through each position in the DataFrame and calls close_position to close them individually.
# Prints the order execution results.


def close_positions(order_type):
    order_type_dict = {
        'buy': 0,
        'sell': 1
    }

    if mt5.positions_total() > 0:
        positions = mt5.positions_get()

        positions_df = pd.DataFrame(positions, columns=positions[0]._asdict().keys())

        if order_type != 'all':
            positions_df = positions_df[(positions_df['type'] == order_type_dict[order_type])]

        for i, position in positions_df.iterrows():
            order_result = close_position(position)

            print('order_result: ', order_result)


In [14]:
# Checks if the current time falls within the allowed trading window (e.g., between 9 AM and 5 PM).
# Uses datetime.now().hour to get the current hour.
# Returns True if within allowed hours, False otherwise.
def check_allowed_trading_hours():
    if 9 < datetime.now().hour < 17:
        return True
    else:
        return False

In [18]:
# Places a market order (immediate execution at the current market price) for the specified symbol, volume, and order type (buy or sell).
# Similar to close_position, uses dictionaries to determine order type and price.
# Defines an order request with details like volume, type, price, and magic number.
# Sends the market order request to the MT5 platform using mt5.order_send.
# Returns the order execution result.

def market_order(symbol, volume, order_type, deviation=20, magic=234000):

    order_type_dict = {
        'buy': mt5.ORDER_TYPE_BUY,
        'sell': mt5.ORDER_TYPE_SELL
    }

    price_dict = {
        'buy': mt5.symbol_info_tick(symbol).ask,
        'sell': mt5.symbol_info_tick(symbol).bid
    }

    # request = {
    #     "action": mt5.TRADE_ACTION_DEAL,
    #     "symbol": symbol,
    #     "volume": volume,  # FLOAT
    #     "type": order_type_dict[order_type],
    #     "price": price_dict[order_type],
    #     "sl": 0.0,  # FLOAT
    #     "tp": 0.0,  # FLOAT
    #     "deviation": deviation,  # INTERGER
    #     "magic": magic,  # INTERGER
    #     "comment": strategy_name,
    #     "type_time": mt5.ORDER_TIME_GTC,
    #     "type_filling": mt5.ORDER_FILLING_IOC,
    # }

    request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol, #str cap(EURUSD)
    "volume": volume, #float lotsize
    "type": mt5.ORDER_TYPE_BUY,
    "price": price_dict[order_type], #float
    "sl": price - 100 * point, #float
    "tp": price + 100 * point, #float
    "deviation": deviation, #int
    "magic": magic, #int
    "comment": strategy_name, #str
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order_result = mt5.order_send(request)
    return(order_result)

In [19]:
# The code starts by initializing the MetaTrader 5 connection and logging in using the imported credentials.
# It enters an infinite loop (while True) that continuously performs the following actions:
# Retrieves account information (login, balance, equity) and prints them.
# Checks the number of open positions (mt5.positions_total).
# Calls check_allowed_trading_hours to see if trading is allowed. If not, all positions are closed using close_positions('all').
# Calculates the fast SMA (10-period) and slow SMA (100-period) for the defined symbol and timeframe using get_sma.
# Implements a simple trading strategy based on the SMA crossover:
# If fast SMA is greater than slow SMA:
# Closes any open sell positions.
# If no positions are open and trading is allowed, a buy market order is placed using market_order.
# If fast SMA is less than slow SMA:
# Closes any open buy positions.
# If no positions are open and trading is allowed, a sell market order is placed


if __name__ == '__main__':
    

    while True:

        num_positions = mt5.positions_total()

        if not check_allowed_trading_hours():
            close_positions('all')

        fast_sma = get_sma(symbol, timeframe, 10)
        slow_sma = get_sma(symbol, timeframe, 100)

        if fast_sma > slow_sma:
            close_positions('sell')

            if num_positions == 0 and check_allowed_trading_hours():
                order_result = market_order(symbol, volume, 'buy')
                print(order_result)

        elif fast_sma < slow_sma:
            close_positions('buy')

            if num_positions == 0 and check_allowed_trading_hours():
                order_result = market_order(symbol, volume, 'sell')
                print(order_result)

        time.sleep(1)

SECTION 4: CANDLESTICK PATTERN RECOGNITION

In [ ]:
# !pip install plotly
# !pip install ipython
# !pip install --upgrade nbformat
# !pip install ipykernel

In [17]:
import plotly.graph_objects as go
from IPython.display import display

In [18]:
# this function retrieves olhc data from MT5 account and return a dataframe
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s')
    
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]

In [19]:
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_M5
start_datetime = datetime(2024, 5, 5)
end_datetime = datetime(2024, 5, 30)

ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
ohlc_df.head()

,time,open,high,low,close
0,2024-05-06 00:00:00,1.07580,1.07596,1.07579,1.07579
1,2024-05-06 00:05:00,1.07579,1.07624,1.07579,1.07618
2,2024-05-06 00:10:00,1.07619,1.07630,1.07610,1.07611
3,2024-05-06 00:15:00,1.07611,1.07624,1.07580,1.07624
4,2024-05-06 00:20:00,1.07624,1.07630,1.07595,1.07596


In [20]:
# visualizing the ohlc data
fig = go.Figure(data=[go.Candlestick(x=ohlc_df['time'],
                open=ohlc_df['open'],
                high=ohlc_df['high'],
                low=ohlc_df['low'],
                close=ohlc_df['close'])])

fig.update_layout(height=600, xaxis_rangeslider_visible=False)
fig.show()

In [22]:
# finding hammer patterns
def find_hammer_pattern(candle):
    candle_range = candle['high'] - candle['low']

    # Check for zero division before proceeding
    if candle_range == 0:
        return False  # No hammer pattern if no price movement
        
    # logic for bullish hammer patterns
    bull_cond1 = candle['close'] > candle['open']
    # check if the hammer pattern is contained in the upper 30% of the candle
    bull_cond2 = (candle['high'] - candle['open']) / candle_range <= 0.3 
    
    
    # logic for bearish hammer patterns
    bear_cond1 = candle['close'] < candle['open']
    # check if the hammer pattern is contained in the lower 30% of the candle
    bear_cond2 = (candle['open'] - candle['low']) / candle_range <= 0.3 

    
    if bull_cond1 and bull_cond2:
        return True
    elif bear_cond1 and bear_cond2:
        return True
    else:
        return False

ohlc_df['hammer_pattern'] = ohlc_df.apply(find_hammer_pattern, axis=1)
ohlc_df

,time,open,high,low,close,hammer_pattern
0,2024-05-06 00:00:00,1.07580,1.07596,1.07579,1.07579,True
1,2024-05-06 00:05:00,1.07579,1.07624,1.07579,1.07618,False
2,2024-05-06 00:10:00,1.07619,1.07630,1.07610,1.07611,False
3,2024-05-06 00:15:00,1.07611,1.07624,1.07580,1.07624,True
4,2024-05-06 00:20:00,1.07624,1.07630,1.07595,1.07596,False
...,...,...,...,...,...,...
5168,2024-05-29 22:40:00,1.08015,1.08015,1.08007,1.08009,False
5169,2024-05-29 22:45:00,1.08009,1.08019,1.08001,1.08002,False
5170,2024-05-29 22:50:00,1.08002,1.08010,1.08000,1.08005,False
5171,2024-05-29 22:55:00,1.08005,1.08018,1.08001,1.08018,False


In [23]:
# visualizing hammers
hammers = ohlc_df[ohlc_df['hammer_pattern'] == True]

fig_hammer = go.Figure(fig)
fig_hammer.update_layout(title='Hammer Candlestick Patterns')

for i, x in hammers.iterrows():
    fig_hammer.add_vline(x=x['time'], line_width=2, line_dash="dash", opacity=0.5, line_color="green")
    
fig_hammer

In [25]:
# finding engulfing patterns
ohlc_df['previous_open'] = ohlc_df['open'].shift(1)
ohlc_df['previous_high'] = ohlc_df['high'].shift(1)
ohlc_df['previous_low'] = ohlc_df['low'].shift(1)
ohlc_df['previous_close'] = ohlc_df['close'].shift(1)

def find_engulfing_pattern(candle):
    
    bull_cond1 = candle['close'] > candle['open'] # bull candle condition
    bull_cond2 = candle['close'] > candle['previous_high'] # engulfment condition
    bull_cond3 = candle['previous_open'] > candle['previous_close'] # previous candle must be a bear candle
    
    bear_cond1 = candle['close'] < candle['open'] # bear candle condition
    bear_cond2 = candle['close'] < candle['previous_low'] # engulfment condition
    bear_cond3 = candle['previous_open'] < candle['previous_close'] # previous candle must be a bull candle
    
    # special condition - engulfing candle body is 1.5 times as long as previous candle range
    # special_cond = abs(candle['open'] - candle['close']) / (candle['previous_high'] - candle['previous_low']) >= 1.5
    previous_range = candle['previous_high'] - candle['previous_low']
    if previous_range == 0:
        special_cond = False  # No engulfing pattern if previous range is zero
    else:
        special_cond = abs(candle['open'] - candle['close']) / previous_range >= 1.5
    
    if bull_cond1 and bull_cond2 and bull_cond3 and special_cond:
        return True
    elif bear_cond1 and bear_cond2 and bear_cond3 and special_cond:
        return True
    else:
        return False

ohlc_df['engulfing_pattern'] = ohlc_df.apply(find_engulfing_pattern, axis=1)
ohlc_df

,time,open,high,low,close,hammer_pattern,previous_open,previous_high,previous_low,previous_close,engulfing_pattern
0,2024-05-06 00:00:00,1.07580,1.07596,1.07579,1.07579,True,NaN,NaN,NaN,NaN,False
1,2024-05-06 00:05:00,1.07579,1.07624,1.07579,1.07618,False,1.07580,1.07596,1.07579,1.07579,True
2,2024-05-06 00:10:00,1.07619,1.07630,1.07610,1.07611,False,1.07579,1.07624,1.07579,1.07618,False
3,2024-05-06 00:15:00,1.07611,1.07624,1.07580,1.07624,True,1.07619,1.07630,1.07610,1.07611,False
4,2024-05-06 00:20:00,1.07624,1.07630,1.07595,1.07596,False,1.07611,1.07624,1.07580,1.07624,False
...,...,...,...,...,...,...,...,...,...,...,...
5168,2024-05-29 22:40:00,1.08015,1.08015,1.08007,1.08009,False,1.08035,1.08036,1.08014,1.08014,False
5169,2024-05-29 22:45:00,1.08009,1.08019,1.08001,1.08002,False,1.08015,1.08015,1.08007,1.08009,False
5170,2024-05-29 22:50:00,1.08002,1.08010,1.08000,1.08005,False,1.08009,1.08019,1.08001,1.08002,False
5171,2024-05-29 22:55:00,1.08005,1.08018,1.08001,1.08018,False,1.08002,1.08010,1.08000,1.08005,False


In [26]:
# visualizing engulfs
engulfs = ohlc_df[ohlc_df['engulfing_pattern'] == True]

fig_engulf = go.Figure(fig)
fig_engulf.update_layout(title='Engulfing Candlestick Patterns')

for i, x in engulfs.iterrows():
    fig_engulf.add_vline(x=x['time'], line_width=2, line_dash="dash", opacity=0.5, line_color="green")
    
fig_engulf

SECTION CANDLESTICK PATTERN TRADING BOT

In [18]:
from time import sleep

In [28]:
def market_order(symbol, volume, order_type, deviation=20, magic=30, stoploss=0.0, take_profit=0.0,
                 strategy_name='Candlestick Bot'):

    order_type_dict = {
        'buy': mt5.ORDER_TYPE_BUY,
        'sell': mt5.ORDER_TYPE_SELL
    }

    price_dict = {
        'buy': mt5.symbol_info_tick(symbol).ask,
        'sell': mt5.symbol_info_tick(symbol).bid
    }

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,  # FLOAT
        "type": order_type_dict[order_type],
        "price": price_dict[order_type],
        "sl": stoploss,  # FLOAT
        "tp": take_profit,  # FLOAT
        "deviation": deviation,  # INTERGER
        "magic": magic,  # INTERGER
        "comment": strategy_name,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,  # mt5.ORDER_FILLING_FOK if IOC does not work
    }

    order_result = mt5.order_send(request)
    return (order_result)

In [29]:
def find_engulfing_pattern(candle):
    bull_cond1 = candle['close'] > candle['open']  # bull candle condition
    bull_cond2 = candle['close'] > candle['previous_high']  # engulfment condition
    bull_cond3 = candle['previous_open'] > candle['previous_close']  # previous candle must be a bear candle

    bear_cond1 = candle['close'] < candle['open']  # bear candle condition
    bear_cond2 = candle['close'] < candle['previous_low']  # engulfment condition
    bear_cond3 = candle['previous_open'] < candle['previous_close']  # previous candle must be a bull candle

    # special condition - engulfing candle body is 1.5 times as long as previous candle range
    if candle['previous_high'] - candle['previous_low'] == 0:
        return False

    special_cond = abs(candle['open'] - candle['close']) / (candle['previous_high'] - candle['previous_low']) >= 1.5

    if bull_cond1 and bull_cond2 and bull_cond3 and special_cond:
        return 'buy'
    elif bear_cond1 and bear_cond2 and bear_cond3 and special_cond:
        return 'sell'
    else:
        return False


In [30]:
def get_engulfing_signal(symbol, timeframe):
    ohlc = mt5.copy_rates_from_pos(symbol, timeframe, 1, 2)
    ohlc_df = pd.DataFrame(ohlc)[['time', 'open', 'high', 'low', 'close']]
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s')

    ohlc_df['previous_open'] = ohlc_df['open'].shift(1)
    ohlc_df['previous_high'] = ohlc_df['high'].shift(1)
    ohlc_df['previous_low'] = ohlc_df['low'].shift(1)
    ohlc_df['previous_close'] = ohlc_df['close'].shift(1)

    ohlc_df['signal'] = ohlc_df.apply(find_engulfing_pattern, axis=1)

    print(ohlc_df)
    return ohlc_df.iloc[-1]['signal']

<!-- Defines strategy parameters:
symbol: The financial instrument to trade (e.g., 'EURUSD').
timeframe: The time frame for price data (e.g., mt5.TIMEFRAME_M1 for 1-minute intervals).
volume: The trading volume for each order (e.g., 0.01 lot).
Sets an initial trading_allowed flag to True.
Enters an infinite loop (while trading_allowed):
Calls get_engulfing_signal to retrieve the latest engulfing pattern signal ('buy', 'sell', or False).
Prints the received signal.
Checks the signal and open positions:
If the signal is 'buy' and there are no open positions, a buy market order is placed using market_order. The loop is then exited by setting trading_allowed to False (assuming only one entry per signal).
Similar logic applies for 'sell' signals.
Introduces a 1-second delay -->

In [31]:

if __name__ == '__main__':
    
    # strategy parameters
    symbol = 'EURUSD'
    timeframe = mt5.TIMEFRAME_M1
    volume = 0.01

    trading_allowed = True
    while trading_allowed:
        # Careful! Loop can open infinite positions!

        signal = get_engulfing_signal(symbol, timeframe)
        print('signal', signal)
        print('---\n')

        if signal == 'buy' and mt5.positions_total() == 0:
            res = market_order(symbol, volume, 'buy')
            trading_allowed = False

        elif signal == 'sell' and mt5.positions_total() == 0:
            res = market_order(symbol, volume, 'sell')
            trading_allowed = False

        sleep(1)

                 time     open     high      low    close  previous_open  \
0 2024-07-04 14:33:00  1.07987  1.07996  1.07987  1.07996            NaN   
1 2024-07-04 14:34:00  1.07996  1.07997  1.07984  1.07994        1.07987   

   previous_high  previous_low  previous_close  signal  
0            NaN           NaN             NaN   False  
1        1.07996       1.07987         1.07996   False  
signal False
---

                 time     open     high      low    close  previous_open  \
0 2024-07-04 14:33:00  1.07987  1.07996  1.07987  1.07996            NaN   
1 2024-07-04 14:34:00  1.07996  1.07997  1.07984  1.07994        1.07987   

   previous_high  previous_low  previous_close  signal  
0            NaN           NaN             NaN   False  
1        1.07996       1.07987         1.07996   False  
signal False
---

                 time     open     high      low    close  previous_open  \
0 2024-07-04 14:33:00  1.07987  1.07996  1.07987  1.07996            NaN   
1 2024-07-04

KeyboardInterrupt: 

In [16]:
# Function to place a market order (replace with your risk management logic)
def place_market_order(symbol, volume, order_type, comment="Algorithmic Trade"):
    """Places a market order on the MT5 platform.

    **Caution:** This is a simplified example. Always implement proper risk management
                 and backtest your strategies before deploying them with real capital.

    Args:
        symbol (str): The financial instrument to trade (e.g., "EURUSD").
        volume (float): The trading volume (e.g., 0.01 lot).
        order_type (str): "buy" or "sell".
        comment (str, optional): A comment to attach to the order. Defaults to "Algorithmic Trade".
    """

    if order_type not in ("buy", "sell"):
        print("Invalid order type. Please specify 'buy' or 'sell'.")
        return

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": mt5.ORDER_TYPE_BUY if order_type == "buy" else mt5.ORDER_TYPE_SELL,
        "type_filling": mt5.ORDER_FILLING_IOC,
        "sl": 0.0010,
        "tp": 0.0020,
        # ... (consider adding stop loss, take profit, and other order parameters)
        "comment": comment
    }

    result = mt5.order_send(request)
    print('Order placed successfully: ', result)
    # if result.retcode != mt5.TRADE_RETCODE_OK:
    #     print(f"Order failed. Error code: {result.retcode}")
    # else:
    #     print(f"Order placed successfully. Order ID: {result.order}")



In [ ]:
if __name__ == "__main__":
    

    # Strategy parameters (replace with your desired values)
    symbol = "EURUSD"
    timeframe = mt5.TIMEFRAME_M1
    volume = 0.01

    # Main loop (replace with your trading logic)
    while True:
        # Simulate market data retrieval (replace with actual data fetching)
        # current_price = ...  # Get the current market price from a reliable source

        # Generate trade signal (replace with your strategy)
        # trade_signal = generate_trade_signals(symbol, timeframe)

        # Place order based on signal (replace with responsible risk management)
        # if trade_signal:
        place_market_order(symbol, volume, 'buy')

        # Introduce a delay to avoid overwhelming the platform (adjust as needed)
        sleep(10)

    # Disconnect from MetaTrader5 when finished
    # mt5.shutdown()

WORK IN PROGRESS

In [ ]:
# Trading Bot

ticker = 'GBPUSD'
qty = 0.01
buy_order_type = mt5.ORDER_TYPE_BUY
sell_order_type = mt5.ORDER_TYPE_SELL
buy_price = mt5.symbol_info_tick("GBPUSD").ask
sell_price = mt5.symbol_info_tick("GBPUSD").bid
sl_pct = 0.05
tp_pct = 0.1
buy_sl = buy_price * (1-sl_pct)
buy_tp = buy_price * (1+tp_pct)
sell_sl = sell_price * (1+sl_pct)
sell_tp = sell_price * (1-tp_pct)

def create_order(ticker, qty, order_type, price, sl, tp):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": qty, # FLOAT
        "type": order_type ,
        "price": price,
        "sl": sl, # FLOAT
        "tp": tp, # FLOAT
        "comment": "Python Open Position",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    order = mt5.order_send(request)
    return order
    
def close_order(ticker, qty, order_type, price):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": qty, # FLOAT
        "type": order_type,
        "position": mt5.positions_get()[0]._asdict()['ticket'], # select the position you want to close
        "price": price,
        "comment": "Close Position",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order = mt5.order_send(request)

create a function that either buys or sells with tp andsl

In [ ]:
import time

# Pine Script

# longCondition = close > high[1]
# shortCondition = close < low[1]
# closelongcondition = close < close[1]
# closeshortcondition = close > close[1]

for i in range(100):
    ohlc = pd.DataFrame(mt5.copy_rates_range('GBPUSD',mt5.TIMEFRAME_M1, datetime(2024,6,21), datetime.now()))
    ohlc['time'] = pd.to_datetime(ohlc['time'],unit='s')
    print(ohlc)

    current_close = list(ohlc[-1:]['close'])[0]
    last_close = list(ohlc[-2:]['close'])[0]
    last_high = list(ohlc[-2:]['high'])[0]
    last_low = list(ohlc[-2:]['low'])[0]
    long_condition = current_close > last_high
    short_condition = current_close < last_low
    closelong_condition = current_close < last_close
    closeshort_condition = current_close > last_close

    already_buy = False
    already_sell = False
    
    try:
        already_sell = mt5.positions_get()[0]._asdict()['type'] == 1
        already_buy = mt5.positions_get()[0]._asdict()['type'] == 0
    except:
        pass

    no_positions = len(mt5.positions_get()) == 0

    if long_condition:
        if no_positions:
            create_order(ticker, qty, buy_order_type, buy_price, buy_sl, buy_tp)
            print('Buy Order Placed')
        if already_sell:
            close_order(ticker,qty,buy_order_type,buy_price)
            print('Sell Position Closed')
            time.sleep(1)
            create_order(ticker, qty, buy_order_type, buy_price, buy_sl, buy_tp)
            print('Buy Order Placed')
    if short_condition:
        if no_positions:
            create_order(ticker, qty, sell_order_type, sell_price, sell_sl, sell_tp)
            print('Sell Order Placed')
        if already_buy:
            close_order(ticker,qty,sell_order_type,sell_price)
            print('Buy Position Closed')
            time.sleep(1)
            create_order(ticker, qty, sell_order_type, sell_price, sell_sl, sell_tp)
            print('Sell Order Placed')

    try:
        already_sell = mt5.positions_get()[0]._asdict()['type'] == 1
        already_buy = mt5.positions_get()[0]._asdict()['type'] == 0
    except:
        pass

    if closelong_condition and already_buy:
        close_order(ticker,qty,sell_order_type,sell_price)
        print('Only Buy Position Closed')
    if closeshort_condition and already_sell:
        close_order(ticker,qty,buy_order_type,buy_price)
        print('Only Sell Position Closed')

    already_buy = False
    already_sell = False
    time.sleep(60)

                     time     open     high      low    close  tick_volume  \
0     2024-06-20 23:00:00  1.26631  1.26631  1.26624  1.26628           61   
1     2024-06-20 23:01:00  1.26628  1.26631  1.26626  1.26630           19   
2     2024-06-20 23:02:00  1.26630  1.26636  1.26630  1.26632           25   
3     2024-06-20 23:03:00  1.26633  1.26633  1.26631  1.26632            3   
4     2024-06-20 23:04:00  1.26632  1.26634  1.26628  1.26630           16   
...                   ...      ...      ...      ...      ...          ...   
12233 2024-07-03 11:14:00  1.26941  1.26975  1.26941  1.26974           76   
12234 2024-07-03 11:15:00  1.26975  1.26981  1.26973  1.26975           84   
12235 2024-07-03 11:16:00  1.26975  1.26976  1.26956  1.26962          106   
12236 2024-07-03 11:17:00  1.26962  1.26964  1.26957  1.26961           71   
12237 2024-07-03 11:18:00  1.26961  1.26962  1.26953  1.26962           62   

       spread  real_volume  
0           3            0  
1    

In [ ]:
# create_order(ticker, qty, buy_order_type, buy_price, buy_sl, buy_tp)

In [ ]:
# close_order = create_order(ticker, qty, sell_order_type, sell_price, sell_sl, sell_tp)
# close_order

LOGIC

In [ ]:
# import time

# # Pine script

# # LongCondition = close > high[1]
# # shortCondition = close < low[1]
# # closeLongCondition = close < close[1]
# # closeShortCondition = close > close[1]

# for i in range(10000000000):
#   ohlc = pd.DataFrame(mt5.copy_rates_range(ticker, interval, datetime(2024, 01,01), datetime.now()))
#   ohlc['time'] = pd.to_datetime(ohlc['time'], unit='s')
#   print(ohlc)


#   current_close = list(ohlc[-1:]['close'])[0]
#   last_close = list(ohlc[-2:]['close'])[0]
#   last_high = list(ohlc[-2:]['high'])[0]
#   last_low = list(ohlc[-2:]['low'])[0]
#   long_condition = current_close > last_high
#   short_condition = current_close < last_low
#   close_long_condition = current_close < last_close
#   close_short_condition = current_close > last_close

#   already_bought = False
#   already_sold = False

#   try:
#     already_sold = mt5.positions_get()[0].asdict()['type'] == 1
#     already_bought = mt5.positions_get()[0].asdict()['type'] == 0
#   except:
#     pass

#   no_positions = len(mt5.positions_get()) == 0

#   if long_condition:
#     if no_positions:
#       create_order(ticker, qty, buy_order_type, buy_price, buy_sl, buy_tp)
#       print('Buy Order Placed')
#     elif already_sold:
#       close_order = create_order(ticker, qty, sell_order_type, sell_price, sell_sl, sell_tp)
#       print('Sell position Closed')
#       time.sleep(1)
#       create_order(ticker, qty, buy_order_type, buy_price, buy_sl, buy_tp)
#       print('Buy Order Placed')
#   if short_condition:
#     if no_positions:
#       create_order(ticker, qty, sell_order_type, sell_price, sell_sl, sell_tp)
#       print('Sell Order Placed')
#     if already_bought:
#       close_order = create_order(ticker, qty, buy_order_type, buy_price, buy_sl, buy_tp)
#       print('Buy position Closed')
#       time.sleep(1)
#       create_order(ticker, qty, sell_order_type, sell_price, sell_sl, sell_tp)
#       print('Sell Order Placed')

#   try:
#     already_sold = mt5.positions_get()[0].asdict()['type'] == 1
#     already_bought = mt5.positions_get()[0].asdict()['type'] == 0
#   except:
#     pass

#   if close_long_condition:
#     if already_bought:
#       close_order = create_order(ticker, qty, sell_order_type, sell_price)
#       print('Only Buy position Closed')
#     if close_short_condition:
#       if already_sold:
#         close_order = create_order(ticker, qty, buy_order_type, buy_price)
#         print('Only Sell position Closed')

#     already_sold = False
#     already_bought = False
#     time.sleep(60)